# Watchify

- ## Watchify is a unique Flask web application that connects your Spotify listening history within the past 30 days with tailored movie and TV show recommendations. Immerse in the personalized visual experience, understanding your music preferences and exploring related cinematic suggestions.
- ## This notebook contains snippets of code used to create Watchify as well as brief explanations in regards to the code.
- ## Download everything in the "static" folder before running the Flask app, so everything can work and be displayed properly.

* ## Data Sources:
  
  - ## Movies: https://www.kaggle.com/datasets/rajugc/imdb-movies-dataset-based-on-genre
  
  - ## TV Shows: https://www.kaggle.com/datasets/payamamanat/imbd-dataset/data
  
  - ## CSV file was created after cleaning up data (find in files)

### Project by: Casey Roosma, Michael Dominguez, Marcela Soriano
### Notebook by: Marcela Soriano



## Libraries Required



In [43]:
!pip install flask-ngrok
!pip install flask_bootstrap
!pip install spotipy

In [44]:
from flask_ngrok import run_with_ngrok
from google.colab import userdata
from flask import Flask, render_template, request, redirect, url_for, session, send_from_directory
from flask_bootstrap import Bootstrap
from os import environ
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import random
import requests
import json
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import os

## Flask Setup and Spotify API

In [45]:
app = Flask(__name__)
app.secret_key = userdata.get('SECRET_KEY')

bootstrap = Bootstrap(app)

genre_count = {}
recommended_tvshow = []
recommended_movie = []

# Add your Spotify API Credentials
SPOTIPY_CLIENT_ID = userdata.get('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = userdata.get('SPOTIPY_CLIENT_SECRET')
SPOTIPY_REDIRECT_URI = userdata.get('SPOTIPY_REDIRECT_URI')

sp_oauth = SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                        client_secret=SPOTIPY_CLIENT_SECRET,
                        redirect_uri=SPOTIPY_REDIRECT_URI,
                        scope=["user-top-read"])

- ### Flask Application Initialization: The Flask framework is used for initializing the web application, providing a foundation for web page creation and request handling.

- ### Session Security Configuration: A secret key for the application is configured, which is crucial for the security of session data in Flask. This key encrypts session cookies and is securely retrieved, indicating an emphasis on maintaining session integrity and security.

- ### Bootstrap Integration for UI Enhancement: Integration with Flask-Bootstrap is implemented. Flask-Bootstrap is an extension that facilitates the use of Bootstrap, a front-end framework, for styling web pages. This ensures the user interface is not only aesthetically pleasing but also responsive.

- ### Data Structure Utilization: Various data structures, such as dictionaries and lists, are employed for storing and managing data related to music genres, as well as recommended TV shows and movies. This likely plays a significant role in the app's recommendation logic.

- ### Spotify API Integration for Music Data: The application is configured to interact with the Spotify API, using Spotify OAuth for secure authentication. This enables the app to access specific Spotify data, like a user's top tracks and artists. The app fetches necessary credentials like the Client ID, Client Secret, and Redirect URI securely, ensuring authenticated and authorized interaction with Spotify.

- ### Defined Scope for Spotify Data Access: The scope for the Spotify API is set to "user-top-read", granting the app the ability to access a user's top Spotify tracks and artists. This feature is essential for functionalities such as generating personalized music recommendations.

## Genre Mapping (Spotify to Movie/TV Show Genre)

In [46]:
#Genre Mapping
genre_mapping = {
    "Romance": {
        'valence': 0.378617,
        'danceability': 0.52972,
        'energy': 0.45700999999999986,
    },
    "Action": {
        'valence': 0.4782686868686866,
        'danceability': 0.610616161616162,
        'energy': 0.7276262626262627,
    },
    "Comedy": {
        'valence': 0.6791019999999999,
        'danceability': 0.6025000000000003,
        'energy': 0.754392,
    },
    "Science Fiction": {
        'valence': 0.318779,
        'danceability': 0.42281799999999997,
        'energy': 0.426963,
    },
    "Horror": {
        'valence': 0.043389655172413784,
        'danceability': 0.2253931034482759,
        'energy': 0.20297327586206892,
    },
    "Family": {
        'valence': 0.39481,
        'danceability': 0.5041180000000001,
        'energy': 0.4358320000000002,
    },
    "Drama": {
        'valence': 0.27197200000000005,
        'danceability': 0.4922300000000001,
        'energy': 0.4408099999999998,
    },
    "Adventure": {
        'valence': 0.4435835164835165,
        'danceability': 0.5584285714285715,
        'energy': 0.6459472527472528,
    },
     "Thriller": {
        'valence': 0.3606849999999999,
        'danceability': 0.5131299999999999,
        'energy': 0.5883509999999998,

        },
     "Biography": {
        'valence': 0.5983500000000002,
        'danceability': 0.60787,
        'energy': 0.7006800000000003,
        },
     "Crime": {
        'valence': 0.5080319999999997,
        'danceability': 0.64384,
        'energy': 0.6495099999999999,
        },
     "Fantasy": {
        'valence': 0.307479,
        'danceability': 0.45702000000000015,
        'energy': 0.4734160000000001,
        }
    }

- ### Valence, danceability and energy are measured by Spotify from 0 to 1.

- ### Valence: A measure of musical positivity. Higher valence indicates a happier, more cheerful sound, while lower valence suggests a more sad or melancholic tone.

- ### Danceability: Describes how suitable a track is for dancing based on tempo, rhythm stability, beat strength, and overall regularity. Higher danceability scores denote music that's easier to dance to.

- ### Energy: A perceptual measure of intensity and activity. Energetic tracks feel fast, loud, and noisy. Higher energy values typically correspond to more intense, lively music.

- ### Each movie/TV genre in the mapping (like "Romance", "Action", "Comedy", etc.) has specific values for these audio features, suggesting a correlation between the musical attributes of Spotify tracks and the moods or themes of these genres. This mapping could be used to recommend movies or TV shows based on a user's Spotify listening preferences, with the idea being that certain musical qualities align with specific cinematic genres. For instance, a high valence in a user's music preference might correlate with a preference for "Comedy" movies, which are also mapped to high valence.

## Personalized Statement

In [47]:
def generate_personalized_statement(valence, danceability, energy):
    # Define custom ranges with two decimal places
    valence_ranges = [(0.00, 0.20), (0.20, 0.40), (0.40, 0.60), (0.60, 0.80), (0.80, 1.00)]
    danceability_ranges = [(0.00, 0.20), (0.20, 0.40), (0.40, 0.60), (0.60, 0.80), (0.80, 1.00)]
    energy_ranges = [(0.00, 0.20), (0.20, 0.40), (0.40, 0.60), (0.60, 0.80), (0.80, 1.00)]

    # Initialize statements
    valence_statement = ""
    danceability_statement = ""
    energy_statement = ""

    # Check Valence
    for lower, upper in valence_ranges:
        if lower <= valence <= upper:
            if valence < 0.20:
                valence_statement = f"Your valence suggests you've been diving deep into the world of sad music. It's time for some virtual hugs!"
            elif valence < 0.40:
                valence_statement = f"Your valence shows a touch of melancholy. Maybe you're composing the soundtrack of a rainy day?"
            elif valence < 0.60:
                valence_statement = f"Your music falls in the 'not too happy, not too sad' range. A well-balanced playlist for life's adventures!"
            elif valence < 0.80:
                valence_statement = f"Your valence indicates a cheerful and positive music taste. Keep rocking those good vibes!"
            else:
                valence_statement = f"Wow, your valence is off the charts! You must be the life of the party with such happy music!"

    # Check Danceability
    for lower, upper in danceability_ranges:
        if lower <= danceability <= upper:
            if danceability < 0.20:
                danceability_statement = f"Your music is not too dance-friendly. Perhaps it's time to join a dance class online?"
            elif danceability < 0.40:
                danceability_statement = f"Your music is moderately danceable. Time to practice your dance moves!"
            elif danceability < 0.60:
                danceability_statement = f"Your music taste is groovy and dance-friendly. Get ready to dance like nobody's watching!"
            elif danceability < 0.80:
                danceability_statement = f"Your music is super danceable! You're a dance floor legend in the making!"
            else:
                danceability_statement = f"Your music is dance-tastic! You're the life of every dance party!"

    # Check Energy
    for lower, upper in energy_ranges:
        if lower <= energy <= upper:
            if energy < 0.20:
                energy_statement = f"Your music is super calm and passive. Are you in relaxation mode or just taking it easy?"
            elif energy < 0.40:
                energy_statement = f"Your music falls within the range of tranquility. Perfect for peaceful moments!"
            elif energy < 0.60:
                energy_statement = f"Your music is moderately energetic. You're keeping it balanced!"
            elif energy < 0.80:
                energy_statement = f"Your music has a good amount of energy. Ready to tackle the day with enthusiasm!"
            else:
                energy_statement = f"Your music is high-energy and intense! It's like a musical energy drink!"

    # Combine the statements
    personalized_statement = valence_statement + "\n" + danceability_statement + "\n" + energy_statement

    return personalized_statement

- ### Valence Analysis: The function assesses the valence (a measure of musical mood) of the user's preferred tracks. Depending on where the valence falls within predefined ranges, a statement is generated to describe the emotional tone of the music, ranging from sad and melancholic to happy and upbeat.

- ### Danceability Analysis: Similarly, it evaluates the danceability of the music. The danceability score determines whether the user prefers music that's not dance-friendly, moderately danceable, groovy, or highly suitable for dancing.

- ### Energy Analysis: The energy level of the music is also analyzed. This results in statements that describe the music's intensity, from calm and passive to energetic and intense.

- ### The function combines these individual statements about valence, danceability, and energy into a single, comprehensive statement that provides a holistic description of the user's musical preferences. This personalized statement can be used to enhance user experience, for example, in music recommendation systems or user profile summaries.

## Pick Movie with All Genres

In [48]:
#Function to pick movie with all genres
def same_genres(genre_choices, genre_csv):
    for x in range(len(genre_csv)):
        for y in genre_choices:
            if y not in genre_csv[x]:
                return False
    return True

- ### Iterating Through Dataset: The function iterates through each entry in the entertainment.csv dataset. Each entry in this dataset presumably contains information about a particular movie or TV show, including its associated genres.

- ### Genre Matching: For each entry in the dataset, the function checks if every genre listed in the user's genre_choices is present. If any genre from genre_choices is not found in the current entry of entertainment.csv, the function immediately returns False, indicating that this entry does not match the complete set of user-preferred genres.

- ### Return Value: If the function completes its iteration through the dataset without returning False, it means that there is at least one entry in entertainment.csv that contains all the genres in genre_choices. In this case, the function returns True.

- ### The function is a filter mechanism, helping to narrow down a list of movies or TV shows to those that align with the complete range of a user's genre preferences. This is particularly useful in a recommendation system where matching content to user preferences is key.

## Index

In [49]:
@app.route('/')
def index():
    return render_template('index.html')

- ### The route and function simply return the rendered HTML content from the index.html template. This means when the root URL is accessed, the Flask app responds by displaying the webpage defined in index.html.

## Login

In [50]:
@app.route('/login')
def login():
    auth_url = sp_oauth.get_authorize_url()
    return render_template('loginpage.html', auth_url=auth_url)

- ### Generating Authorization URL: It calls sp_oauth.get_authorize_url() to generate an authorization URL for Spotify's OAuth service. This URL is used to authenticate users and grant your application access to their Spotify data, based on the permissions defined in your OAuth setup.

- ### Rendering Login Page: The function then returns a rendered HTML template (loginpage.html), passing the generated auth_url to it. This page likely contains a login mechanism or a link/button which directs the user to the Spotify authentication page using the provided auth_url.

- ### The route serves as the entry point for user authentication with Spotify, redirecting them to login via Spotify and authorize your application to access their Spotify data as per the defined scopes.

## Callback

In [51]:
@app.route('/callback')
def callback():
    code = request.args.get('code')
    if not code:
        # Handle the case where the code is missing
        return "Error: No code provided.", 400
    try:
        token_info = sp_oauth.get_access_token(code, check_cache=False)
        session['token'] = token_info['access_token']
    except Exception as e:
        # Log the exception for debugging
        print(f"Error retrieving access token: {e}")
        return "Error in token retrieval.", 500

    # Initialize the Spotipy client with the access token
    global sp
    sp = spotipy.Spotify(auth=session['token'])
    return render_template('loading.html')

- ### Retrieving Authorization Code: The function starts by attempting to retrieve an authorization code (code) from the query parameters of the request. This code is provided by the Spotify OAuth service after a user successfully logs in and authorizes your application.

- ### Handling Missing Code: If the code is not found in the request (indicating an issue in the OAuth flow), the function returns an error message and a 400 HTTP status code, indicating a bad request.

- ### Token Retrieval: If the code is present, the function tries to exchange this code for an access token using sp_oauth.get_access_token(code). This access token is necessary for making authenticated requests to Spotify's API.

- ### Error Handling: If there's an error during token retrieval, such as a network issue or an invalid code, the function catches the exception, logs it for debugging, and returns an error message with a 500 HTTP status code, indicating a server error.

- ### Session Token Storage: Upon successful retrieval, the access token is stored in the Flask session (session['token']). This allows your application to use the token in subsequent requests to the Spotify API.

- ### Spotify Client Initialization: The function then initializes the Spotipy client (sp) with the retrieved access token. This client is used for interacting with the Spotify API.

- ### Redirecting to Loading Page: Finally, the function renders and returns loading.html, likely a page to inform the user that their request is being processed or to handle further navigational steps.

- ### This route is a crucial part of the OAuth process, handling the exchange of the authorization code for an access token and preparing the application for further interactions with the Spotify API.

## Display History (Top 3 Spotify Genres over past 30 Days)

In [52]:
@app.route('/history')
def display_history():
    if 'token' not in session:
        return redirect(url_for('login'))

    sp = spotipy.Spotify(auth=session['token'])

    # This function or variable definition should be before its usage.
    top_5_genres = sp.current_user_top_artists(limit=5)['items']

    current_user = sp.current_user()
    # Initialize user_metrics
#-------------------------- metrics    -----------------------------------------------------------
    #Getting the avergage metrics for the user
    user_metrics = {"valence": 0, "danceability": 0, "energy": 0}
    data = {'Genre': [], 'Valence': [], 'Danceability': [], 'Energy': [], 'Count': []}

    #Getting top artists genres
    top_artists = sp.current_user_top_artists(limit=30, time_range="short_term")['items']
    count = 0
    pre_data = {}

    artist_count = 0
    while len(pre_data) < 5 and artist_count < len(top_artists):
        if len(top_artists[artist_count]['genres']) > 0 and top_artists[artist_count]['genres'][0] not in pre_data:
            pre_data[top_artists[artist_count]['genres'][0]] = {'Valence': 0.0, 'Danceability': 0.0, 'Energy': 0.0, 'Count': 0}
        artist_count += 1


    print("DATA::::", pre_data)
    tracks = sp.current_user_top_tracks(time_range='short_term', limit=30)['items']
    for track in tracks:
        song_id = track['id']
        song_metrics = sp.audio_features(song_id)[0]
        if song_metrics is not None:
            user_metrics["valence"] += song_metrics['valence']
            user_metrics["danceability"] += song_metrics['danceability']
            user_metrics["energy"] += song_metrics['energy']
            count += 1
        if len(sp.artist(track['album']['artists'][0]['id'])['genres']) > 0:
            artist_gen = sp.artist(track['album']['artists'][0]['id'])['genres'][0]
            if artist_gen in pre_data:
                pre_data[artist_gen]["Valence"] += song_metrics['valence']
                pre_data[artist_gen]["Danceability"] += song_metrics['danceability']
                pre_data[artist_gen]["Energy"] += song_metrics['energy']
                pre_data[artist_gen]["Count"] += 1
    if count > 0:
        user_metrics["valence"] = user_metrics["valence"] / count
        #data['Valence'] = user_metrics["valence"]
        user_metrics["danceability"] = user_metrics["danceability"] / count
       # data['Danceability'] = user_metrics["danceability"]
        user_metrics["energy"] = user_metrics["energy"] / count
        #data["Energy"] = user_metrics["energy"]

#Now finds which genre is closest to the users metrics
    global genre_td
    genre_td = {
        "Romance": {},
        "Action": {},
        "Comedy": {},
        "Science Fiction": {},
        "Horror": {},
        "Family": {},
        "Drama": {},
        "Adventure": {},
        "Thriller": {},
        "Biography": {},
        "Crime": {},
        "Fantasy": {}
}
    global Genre_choice
    Genre_choice = []
    best_genre = None
    best_diff = float('inf')
    for genre, genre_metrics in genre_mapping.items():
        valence_diff = abs(user_metrics["valence"] - genre_metrics["valence"])
        danceability_diff = abs(user_metrics["danceability"] - genre_metrics["danceability"])
        energy_diff = abs(user_metrics["energy"] - genre_metrics["energy"])

        total_diff = valence_diff + danceability_diff + energy_diff
        genre_td[genre] = total_diff

        if total_diff <= best_diff:
            best_genre = genre
            best_diff = total_diff

    if best_genre not in Genre_choice:
        Genre_choice.append(best_genre)

    for genre in genre_td:
        if abs(genre_td[genre] - best_diff) <= 0.02:
            if genre not in Genre_choice:
                Genre_choice.append(genre)

    statement = generate_personalized_statement(user_metrics['valence'], user_metrics['danceability'], user_metrics['energy'])
    print(statement)
#----------------------Adding To The Data-----------------------------------------------

    keys_to_delete = [key for key in pre_data if pre_data[key]["Count"] == 0]

    for key in keys_to_delete:
        del pre_data[key]

    for x in pre_data:
        for y in pre_data[x]:
            if y != "Count":
             pre_data[x][y] = pre_data[x][y]/ pre_data[x]['Count']

    for x in pre_data:
        data['Genre'].append(x)
        for y in pre_data[x]:
            if y != "Count":
                data[y].append(pre_data[x][y])
    data['Count'] = count

#--------------------MATPLOT-----------------
    df = pd.DataFrame(data)
    melted_df = df.melt(id_vars=['Genre', 'Count'], value_vars=['Valence', 'Danceability', 'Energy'],
                    var_name='Feature', value_name='Average')

    # Increase font size globally for the plot
    plt.rcParams.update({'font.size': 18})  # Increase the base font size

    try:
        # Plotting the graph with a larger figure size
        plt.figure(figsize=(20, 12))  # Increased figure size
        barplot = sns.barplot(x='Genre', y='Average', hue='Feature', data=melted_df, palette=['#1db954', '#191414', '#ababab'])

        # Customize the plot with larger font sizes
        plt.title('Top 3 Spotify Genres and Audio Features (Last 30 Days)', fontsize=20)  # Increased title font size
        plt.ylabel('Average Feature Value', fontsize=18)  # Increased y-axis label font size
        plt.xlabel('Genre', fontsize=18)  # Increased x-axis label font size
        plt.xticks(rotation=45)
        plt.legend(title='Feature', fontsize=18)  # Increased legend font size

        # Remove y-axis labels on the left
        plt.gca().tick_params(labelleft=False)

        # Capitalize the first letter of each genre and annotate values
        for p in barplot.patches:
            plt.annotate(format(p.get_height(), '.2f'),
                     (p.get_x() + p.get_width() / 2., p.get_height()),
                     ha = 'center', va = 'center',
                     xytext = (0, 9),
                     textcoords = 'offset points')

        # Update the x-axis labels to capitalize genres and ensure they are unique
        unique_genres = melted_df['Genre'].unique()
        plt.xticks(range(len(unique_genres)), [label.capitalize() for label in unique_genres])

        # Save the plot as a PNG file in a BytesIO object
        img = BytesIO()
        plt.savefig(img, format='png', bbox_inches='tight')
        img.seek(0)  # Rewind the file
        plt.clf()  # Clear the figure to free memory

        # Save the plot to the 'static' directory
        img_path = os.path.join('static', 'user_plot.png')
        with open(img_path, 'wb') as f:
            f.write(img.getvalue())

        # Pass the image path to the template for display
    except:
        print("Error: DataFrame Empty!")
    return render_template('displayhistory.html', img_path=img_path, username=current_user['display_name'], personalized_statement=statement)


#--------------------MATPLOT-----------------
@app.route('/download_plot')
def download_plot():
    # Provide a route to download the plot
    img_path = os.path.join('static', 'user_plot.png')
    return send_from_directory(directory='static', path='user_plot.png', as_attachment=True, download_name='YourSpotifyJourney.png')


- ### Token Verification: Checks if the Spotify access token is present in the session. If not, it redirects the user to the login page for authentication.

- ### Spotify API Interaction: Uses the access token to create a Spotify client and fetches the current user's top artists and tracks. This data is used to analyze the user's music preferences.

- ### Data Aggregation: The function collects various metrics like valence, danceability, and energy from the user's top tracks. These metrics are averaged to represent the user's overall music taste.

- ### Genre Analysis: It also analyzes the genres associated with the top tracks and artists, contributing to a more nuanced understanding of the user's preferences.

- ### Genre-to-Mood Mapping: The function compares these metrics against predefined genre profiles to determine which genres closely match the user's listening habits.

- ### Personalized Statement Generation: A personalized statement is created based on the user's musical metrics, providing a narrative summary of their music taste.

- ### Data Visualization: The aggregated data is visualized in a bar plot using Matplotlib and Seaborn, showcasing the average values of audio features across different genres.

- ### Rendering the Result: Finally, the route renders displayhistory.html, passing the path to the generated plot and other user-specific data. This displays a comprehensive history and analysis of the user's music preferences.

- ### Additionally, a /download_plot route is provided, enabling users to download the generated plot as a PNG file, enhancing the interactive experience of the application.

## Movie/TV Show Recommendation

In [53]:
@app.route('/recommendation', methods=['GET', 'POST'])
def recommendation():
    choice = ""
    token = session.get('token')
    if not token:
        return redirect(url_for('login'))

    #User picks which media they want and we find the reccomended movie
    if request.method == 'POST':
        choice = request.form.get('choice')
        df = pd.read_csv('entertainment.csv')
        df_filtered = ""
        if choice == "movie":
            if len(Genre_choice) > 1:
                df_filtered = df[df['genres'].notna() & (df['media'] == "movie")]
                for gen in Genre_choice:
                    df_filtered = df_filtered[(df_filtered['genres'].str.contains(gen))]
                if len(df_filtered[df_filtered['genres'].str.count(',') + 1 == len(Genre_choice)]) > 1:
                    df_filtered = df_filtered[df_filtered['genres'].str.count(',') + 1 == len(Genre_choice)]
            if len(df_filtered) == 0 or len(Genre_choice) == 1:
                df_filtered = df[df['genres'].notna() & (df['media'] == "movie") & df['genres'].str.contains(Genre_choice[0])]
            recommended_movie = df_filtered.sample().iloc[0]
            return render_template('displayrecommendation.html', recommended_movie=recommended_movie, choice=choice, genre_choice=Genre_choice[0])
        elif choice == "tvshow":
            if len(Genre_choice) > 1:
                df_filtered = df[df['genres'].notna() & (df['media'] == "tv")]
                for gen in Genre_choice:
                    df_filtered = df_filtered[(df_filtered['genres'].str.contains(gen))]
            if len(df_filtered) == 0 or len(Genre_choice) == 1:
                df_filtered = df[df['genres'].notna() & (df['media'] == "tv") & df['genres'].str.contains(Genre_choice[0])]
            recommended_show = df_filtered.sample().iloc[0]
            return render_template('displayrecommendation.html', recommended_show=recommended_show, choice=choice)
    return render_template('displayrecommendation.html', choice=choice)

- ### Token Verification: Initially, the function checks if a Spotify access token exists in the session. If not, it redirects the user to the login page for authentication.

- ### Handling GET and POST Requests: This route supports both GET and POST methods. The GET method displays the recommendation page, while the POST method is used when the user submits their choice.

- ### User's Choice Processing: When a POST request is made (indicating the user has made a choice between 'movie' or 'tvshow'), the function fetches this choice from the form data.

- ### Data Filtering for Recommendations: The application reads from a dataset (entertainment.csv), which contains information about various movies and TV shows. Based on the user's choice and their preferred genres (stored in Genre_choice), it filters this dataset to find matching media.

- ### If the user chooses "movie", it filters the dataset for movies matching the user's genre preferences.
- ### If "tvshow" is chosen, it similarly filters for TV shows.
- ### Random Recommendation Selection: From the filtered list, a random movie or TV show is selected as the recommendation.

- ### Rendering the Recommendation: Finally, the function renders displayrecommendation.html, passing details of the recommended movie or TV show, along with the user's initial choice.

- ### This route essentially bridges the user's music taste with film and TV content, suggesting visual entertainment options that align with their auditory preferences.

## Run Flask App

In [40]:
run_with_ngrok(app)  # Start ngrok when the app is run
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://095e-34-29-196-84.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


- ### Click + CTRL on '*Running on http://127.0.0.1:500' to run Flask app effectively.
- ### Make sure to download "satic" folder and entertainment.csv file to make sure Watchify runs on your local host.